In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [51]:
# Вот здесь – эмитенты, там нужно отобрать именно страны
data_entity =  export_from_RISKCUSTOM('select * from "RISKACCESS"."umdapCBondsEmitents"')
# Вот здесь – сами рейтинги, нас интересуют long term foreign currency от трех агентств: Moody’s, Standarg & Poors (S&P), и Fitch Ratings.
data_rating = export_from_RISKCUSTOM('''select * from "RISKACCESS"."umdapCBondsEmitentRatings" where "agency_name_eng" in ('S&P Global Ratings', 'Moody''s Investors Service', 'Fitch Ratings') ''')
data_old = pd.read_excel('Country retings check.xlsx')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [28]:
Countries_list = data_old.Countries.tolist()
len(Countries_list)

109

In [6]:
data_rating.columns

Index(['##batch_id', '##deleted', '##inserted', '##origin_name', 'id', 'bik',
       'branch_id', 'branch_name_eng', 'branch_name_ita', 'branch_name_pol',
       'branch_name_rus', 'country_id', 'country_name_eng', 'country_name_ita',
       'country_name_pol', 'country_name_rus', 'country_region_id',
       'country_subregion_id', 'code_of_emitent', 'email',
       'emitents_id_absorption', 'emitent_address_eng', 'emitent_address_ita',
       'emitent_address_pol', 'emitent_address_rus', 'emitent_inn',
       'emitent_inn_ua', 'emitent_kpp', 'emitent_lei', 'emitent_spv', 'fax',
       'full_inn', 'full_name_eng', 'full_name_ita', 'full_name_pol',
       'full_name_rus', 'legal_address_eng', 'legal_address_ita',
       'legal_address_pol', 'legal_address_rus', 'name_eng', 'name_ita',
       'name_pol', 'name_rus', 'ogrn', 'ogrn_date', 'ogrn_first_date', 'okpo',
       'phone', 'regsn', 'reg_form_id', 'reg_form_name_eng',
       'reg_form_name_ita', 'reg_form_name_pol', 'reg_form_name_r

In [29]:
data_entity.loc[data_entity['country_name_eng'].isin(Countries_list),:]

,##batch_id,##deleted,##inserted,##origin_name,id,bik,branch_id,branch_name_eng,branch_name_ita,branch_name_pol,...,registration_country_pol,registration_country_ita,id_group,group_name_rus,is_emitent,cik,sic,loadEventId,sic_name_eng,is_country
0,27444761@WHPROD,N,2023-06-12 08:00:19,WHFLEX,2801,044525105,27,Banks,Banks,Banks,...,Rosja,Russia,117.0,Commerzbank,0,None,0,46390,None,0
1,31308103@WHPROD,N,2023-11-20 08:00:32,WHFLEX,13185,None,27,Banks,Banks,Banks,...,India,India,2635.0,Axis Bank,1,None,6020,67971,Commercial banks,0
2,27152730@WHPROD,N,2023-05-29 08:00:19,WHFLEX,377,None,27,Banks,Banks,Banks,...,Kazakhstan,Kazakhstan,NaN,None,1,None,6021,45563,NATIONAL COMMERCIAL BANKS,0
3,27891885@WHPROD,N,2023-07-03 08:00:22,WHFLEX,16053,None,27,Banks,Banks,Banks,...,Mexico,Mexico,949.0,BBVA,1,None,6021,47657,NATIONAL COMMERCIAL BANKS,0
4,31308103@WHPROD,N,2023-11-20 08:00:32,WHFLEX,66741,None,27,Banks,Banks,Banks,...,China,China,2843.0,Industrial Bank,1,None,6029,67971,"COMMERCIAL BANKS, NEC",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,41458189@WHPROD,N,2024-10-28 08:00:50,WHFLEX,95553,None,27,Banks,Banks,Banks,...,India,India,NaN,None,1,None,6020,278041,Commercial banks,0
2025,41458189@WHPROD,N,2024-10-28 08:00:50,WHFLEX,95685,None,27,Banks,Banks,Banks,...,Argentina,Argentina,110.0,HSBC,1,None,6719,278041,Holding companies,0
2026,41458189@WHPROD,N,2024-10-28 08:00:50,WHFLEX,96777,None,15,Miscellaneous Transportation Services,Transportation,Transport i logistyka,...,Switzerland,Switzerland,NaN,None,1,None,0,278041,None,0
2027,41458189@WHPROD,N,2024-10-28 08:00:50,WHFLEX,855,044030790,27,Banks,Banks,Banks,...,Rosja,Russia,3205.0,Банк Санкт-Петербург,1,None,6029,278041,"COMMERCIAL BANKS, NEC",0


In [161]:
# выбор нужного рейтинга
data_rating_LT_foreign = data_rating[data_rating.scale_name_eng.str.lower().apply(lambda x: 'foreign' in x.lower())]
# после выбора нужного рейтинга осталось 3 организации
assert data_rating_LT_foreign.agency_name_eng.nunique() == 3
# остался только долгосрочный рейтинг
assert ([x for x in data_rating_LT_foreign.scale_name_eng.unique() if 'LT' in x] == data_rating_LT_foreign.scale_name_eng.unique()).all() == np.True_
# выбор стран в качестве эмитента
data_rating_LT_foreign_countries = data_rating_LT_foreign[data_rating_LT_foreign.emission_emitent_name_eng.isin(Countries_list)]
# выбор максимальной даты
data_rating_LT_foreign_countries['max_updating_date'] = data_rating_LT_foreign_countries.groupby(dropna=False, by=['emission_emitent_name_eng','agency_name_eng'])['updating_date'].transform(max)
data_rating_LT_foreign_countries_date = data_rating_LT_foreign_countries[data_rating_LT_foreign_countries.max_updating_date==data_rating_LT_foreign_countries.updating_date]

# pivot
data_rating_work = data_rating_LT_foreign_countries_date[['emission_emitent_name_eng','agency_name_eng','updating_date','forecast_name_eng', 'scale_point_id','scale_point_name']]
data_rating_work_pivot = data_rating_work.pivot_table(index='emission_emitent_name_eng', columns='agency_name_eng', values=['updating_date','forecast_name_eng','scale_point_name'], aggfunc=lambda x: list(x)[0]).reset_index()
# rename columns
columns_list = data_rating_work_pivot.columns.tolist()
new_names_dict = {}
for x in columns_list:
    if x[0] == 'forecast_name_eng':
        new_names_dict[x] = str(x[1].split(' ')[0]) + '_outlook'
    if x[0] == 'scale_point_name':
        new_names_dict[x] = str(x[1].split(' ')[0])
    if x[0] == 'updating_date':
        new_names_dict[x] = str(x[1].split(' ')[0]) + '_date'
data_rating_work_pivot.columns = ['Countries'] + list(new_names_dict.values())

# mergw old data 
data_new = data_old.loc[:,:'Countries'].merge(data_rating_work_pivot,how='left', on='Countries').sort_values(['Fitch_date',	'Moody\'s_date',	'S&P_date'])


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9376\4293969401.py:10: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data_rating_LT_foreign_countries['max_updating_date'] = data_rating_LT_foreign_countries.groupby(dropna=False, by=['emission_emitent_name_eng','agency_name_eng'])['updating_date'].transform(max)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9376\4293969401.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rating_LT_foreign_countries['max_updating_date'] = data_rating_LT_foreign_countries.groupby(dropna=False, by=['emission_emitent_n

In [164]:
data_rating_LT_foreign.scale_name_eng.unique()

array(['LT Int. Scale (foreign curr.)', 'LT- foreign currency',
       'Foreign Currency LT'], dtype=object)

In [162]:
data_new

,Code,Countries,Fitch_outlook,Moody's_outlook,S&P_outlook,Fitch,Moody's,S&P,Fitch_date,Moody's_date,S&P_date
47,ML,Mali,None,Stable,None,Withdrawn,Caa2,NR,2016-08-24 09:50:52,2024-06-12 09:28:24,2021-03-17 16:35:58
58,RU,Russia,None,None,None,Withdrawn,Withdrawn,NR,2022-03-27 16:48:58,2022-03-31 16:36:32,2022-04-11 07:42:47
54,BY,Belarus,None,Stable,None,Withdrawn,C,NR,2023-10-09 09:10:58,2023-06-05 09:55:41,2023-09-11 10:00:02
0,AU,Australia,Stable,Stable,Stable,AAA,Aaa,AAA,2023-11-10 08:47:55,2024-10-10 09:12:59,2024-09-26 10:09:05
88,TH,Thailand,Stable,Stable,Stable,BBB+,Baa1,BBB+,2023-11-14 09:51:42,2024-04-12 10:24:00,2023-11-29 09:08:06
...,...,...,...,...,...,...,...,...,...,...,...
97,YE,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
98,LR,Liberia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
99,MH,Marshall Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
100,NaN,Togo,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [163]:
data_new.to_excel(f'{str(date.today())}_Country_retings_check.xlsx',index=False)